In [3]:
#distance of first wall cell - 2.246E-5m when y+ = 0.5
#y+ = 5 dy = 2.25E-4m
#10 layers of wall cells near the wall
# Grid numbers in x,y directions
#h5 x:70 y:60 convergency time step:872
#h10 x:90 y:80 convergency time step:1743
#h15 x:135 y:100 convergency time step:2995
#h20 x:180 y:120 convergency time step:4265
import numpy as np
import math
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
head ='/home/haoyan/Desktop/ML-CFD/CASE/PHill/Grid-iretation-independency/'
file = ['h5','h8','h10']
i = [70,93,140]
j = [65,83,120]
var = ['U','k','omega']
steps = [800,1700,2900,4200]
#x-coordinate:Cx,y-coordinate:Cy,Velocity:U,k:k,omega:omega
#epsilon = 0.09 * omega * k

In [5]:
def reading_field(head,name,n_package,file_name,i,j):#Reading the raw vector data
    file = head +name +'/'+ n_package + '/' + file_name
    raw_data = []
    number = 23 + (i * j)
    with open(file) as f:
        for line in f:
            line = line.strip('\n')
            raw_data.append(line)
    data_str = (raw_data[23:number])
    data  = [float(i) for i in data_str]
    data = np.array(data)
    return data

def reading_U(head,name,n_package,file_name,i,j): # Reading second-rank tensor in OpenFoam field
    file = head +name +'/'+ n_package + '/' + file_name
    raw_data = []
    number = (i * j)
    with open(file) as f:
        for line in f:
            line = line.strip('\n ()')
            raw_data.append(line)
    data_str = raw_data[23:(23+number)]
    tensors = np.zeros((number,3))
    for i in range(number):
        element = np.array([float(i) for i in data_str[i].split()])
        tensors[i] = element
    U = tensors[:,0]
    V = tensors[:,1]
    return U,V

def grid_plot(x,y):#Drawing the scatter of mesh centres
    plt.figure(figsize=(90,30.35))
    plt.scatter(x,y,color='k')
    plt.xlabel('x')
    plt.ylabel('y')
    
def Epsilon(omega,k):
    ep = []
    for i in range(len(omega)):
        ep.append(0.09*omega[i]*k[i])
    #ep = np.array(ep)
    return ep
def Delta(U,U_):
    D = abs(max(U_) - min(U_))
    delta = []
    for i in range(len(U)):
        delta.append(abs(U[i]-U_[i])/D)
    #delta = np.array(delta)
    return delta

In [11]:
def criteria_b(step1,step2,head,file,i,j):
    n = step1
    var = ['U','k','omega']
    #step = 10700
    u = []
    v = []
    K = []
    e = []
    o = []
    while (n<=(step2-100)):
        U,V = reading_U(head,file,str(n),var[0],i,j)
        k = reading_field(head,file,str(n),var[1],i,j)
        omega = reading_field(head,file,str(n),var[2],i,j)
        epsilon = Epsilon(omega,k)
        n+=100
        U_,V_ = reading_U(head,file,str(n),var[0],i,j)
        k_ = reading_field(head,file,str(n),var[1],i,j)
        omega_ = reading_field(head,file,str(n),var[2],i,j)
        epsilon_ = Epsilon(omega_,k_)
        delta_U = Delta(U,U_)
        delta_V = Delta(V,V_)
        delta_k = Delta(k,k_)
        delta_omega = Delta(omega,omega_)
        delta_epsilon = Delta(epsilon,epsilon_)
        u.append(max(delta_U))
        v.append(max(delta_V))
        K.append(max(delta_k))
        o.append(max(delta_omega))
        #e.append(max(delta_epsilon))
    return u,v,K,o

In [7]:
#h10
u,v,k,o = criteria_b(13100,14000,head,file[2],i[2],j[2])
print(max(u),'\n',max(v),'\n',max(k),'\n',max(o))

7.37421252632816e-06 
 7.612363239150163e-06 
 2.422528569321521e-06 
 9.990672506918033e-08


In [9]:
#h8
u,v,k,o = criteria_b(9100,10000,head,file[1],i[1],j[1])
print(max(u),'\n',max(v),'\n',max(k),'\n',max(o))

7.417764804979926e-06 
 2.387827463402143e-06 
 1.237655065833214e-06 
 1.0022112293690731e-07


In [13]:
#h5
u,v,k,o = criteria_b(9100,10000,head,file[0],i[0],j[0])
print(max(u),'\n',max(v),'\n',max(k),'\n',max(o))

7.47530532889513e-06 
 2.3147803378957598e-06 
 1.2695243061499072e-06 
 1.0043531954776389e-07


4.4468730568486365e-05 
 0.0001748123310508464 
 0.00010257230017872836 
 3.127876116466404e-07


In [2]:
1.21/28

0.04321428571428571